In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from utilities import read_dataset_folder, describe, statistics

import random
import numpy as np
import ast
import dis
import types

def instructions(co):
    bytecode = dis.Bytecode(co)
    iset = []
    for instr in bytecode:
        iset.append(instr.opname)
        if isinstance(instr.argval, types.CodeType):
            iset += instructions(instr.argval)
    return iset

def vectorize(codes):
    isets = []
    for code in codes:
        tree = ast.parse(code)
        co = compile(tree, '', 'exec')
        iset = instructions(co)
        isets.append(iset)
    ops = set(i for iset in isets for i in iset)
    freqs = {op: sum(op in iset for iset in isets) for op in ops}
    print(f'Dim (opcode count): {len(ops)}')
    vecs = []
    for iset in isets:
        total = len(iset)
        vec = []
        for op in ops:
            tf = len([i for i in iset if i == op]) / total
            idf = np.log(len(isets) / freqs[op])
            vec.append(tf)
        vecs.append(vec)
    return vecs

def dataset(folder='data', count=2000):
    random.seed(42)
    programs = random.sample(read_dataset_folder(folder), count)
    print(f'Total programs: {len(programs)}')
    vectors = vectorize([code for code, _ in programs])
    x = np.array(list(vectors), dtype=np.float32)
    y = np.array([klass for _, klass in programs], dtype=int).reshape((len(programs), 1))
    return None, x, y

describe(dataset)

Total programs: 100
Dim (opcode count): 73
Components: 73.
Non-zero components: 15.
Zero components: 58.


**Plots 100-1000**

In [2]:
stats = statistics(dataset, KNeighborsClassifier, {
    'n_neighbors': [4],
    'weights': ('distance',),
    'metric': ('manhattan',)
}, 100, 1000 + 100, 100)

Total programs: 100
Dim (opcode count): 73
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s
[CV] END ..metric=manhattan, n_neighbors=4, weights=distance; total time=   0.0s

{'metric': 'manhattan', 'n_neighbors': 4, 'weights': 'distance'}
{'accuracy': (0.8, 0.095),
 'f1_macro': (0.762, 0.121),
 'f1_weighted': (0.78, 0.101),
 'fit_time': (0.0, 0.0),
 'precision_macro': (0.775, 0.128),
 'precision_weighted': (0.802, 0.112),
 'recall_macro': (0.787, 0.115),
 'recall_weighted': (0.8, 0.095),
 'score_time': (0.003, 0.0)}
Total programs: 200
Dim (opcode count): 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..metric=manhattan, n_neighbors=4

In [3]:
stats = statistics(dataset, SVC, {
    'C': [30],
    'kernel': ('rbf',),
}, 100, 1000 + 100, 100)

Total programs: 100
Dim (opcode count): 73
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s
[CV] END ...................................C=30, kernel=rbf; total time=   0.0s

{'C': 30, 'kernel': 'rbf'}
{'accuracy': (0.85, 0.055),
 'f1_macro': (0.821, 0.081),
 'f1_weighted': (0.845, 0.056),
 'fit_time': (0.001, 0.0),
 'precision_macro': (0.851, 0.075),
 'precision_weighted': (0.881, 0.047),
 'recall_macro': (0.831, 0.075),
 'recall_weighted': (0.85, 0.055),
 'score_time': (0.003, 0.0)}
Total programs: 200
Dim (opcode count): 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=30, kernel=rbf; total time=  

In [4]:
stats = statistics(dataset, RandomForestClassifier, {
    'n_estimators': [300],
    'max_depth': [40],
}, 100, 1000 + 100, 100)

Total programs: 100
Dim (opcode count): 73
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s
[CV] END .....................max_depth=40, n_estimators=300; total time=   0.2s

{'max_depth': 40, 'n_estimators': 300}
{'accuracy': (0.84, 0.073),
 'f1_macro': (0.782, 0.077),
 'f1_weighted': (0.822, 0.073),
 'fit_time': (0.182, 0.006),
 'precision_macro': (0.795, 0.063),
 'precision_weighted': (0.84, 0.056),
 'recall_macro': (0.803, 0.082),
 'recall_weighted': (0.84, 0.073),
 'score_time': (0.015, 0.0)}
Total programs: 200
Dim (opcode count): 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .....................max_depth=40, n_estimators=300; 

In [5]:
stats = statistics(dataset, MLPClassifier, {
    'activation': ['relu'],
    'learning_rate_init': [0.001],
}, 100, 1000 + 100, 100)

Total programs: 100
Dim (opcode count): 73
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s
[CV] END ..........activation=relu, learning_rate_init=0.001; total time=   0.1s

{'activation': 'relu', 'learning_rate_init': 0.001}
{'accuracy': (0.57, 0.093),
 'f1_macro': (0.405, 0.102),
 'f1_weighted': (0.467, 0.104),
 'fit_time': (0.051, 0.001),
 'precision_macro': (0.377, 0.108),
 'precision_weighted': (0.43, 0.111),
 'recall_macro': (0.491, 0.093),
 'recall_weighted': (0.57, 0.093),
 'score_time': (0.002, 0.0)}
Total programs: 200
Dim (opcode count): 82
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..........activation=relu, learning_rate